<a href="https://colab.research.google.com/github/Droppicode/classify_transactions/blob/main/classify_transactions_better_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
import numpy as np
import random
import json

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', 1000)

# Abre base de dados

with open("base_dados.json", "r") as json_file:
    base_dados = json.load(json_file)

categorias = list(base_dados.keys())
# ['SUPERMERCADO', 'RESTAURANTE', 'CASA', 'LOJAS DE DEPARTAMENTO', 'ROUPAS', 'COMBUSTIVEL', 'MANUTENCAO', 'MEDICAMENTOS', 'COSMETICOS', 'PLANO DE SAUDE', 'SERVICOS', 'ASSINATURAS', 'LAZER', 'LUZ', 'AGUA', 'GAS', 'IMPOSTOS', 'TAXAS', 'INVESTIMENTOS', 'EDUCACAO']

# Sujeira para as transacoes

sujeira_prefixos = ['COMPRA', 'PGTO', 'DEBITO', 'CREDITO', 'PIX', 'TED', 'DOC', 'EXTRATO', 'COMPRA ELO', 'COMPRA VISA', 'ELO', 'VISTA']
sujeira_sufixos = ['SP', 'RJ', 'BH', 'CURITIBA', 'MATRIZ', 'FILIAL', 'S.A.', 'LTDA', 'PAGAMENTOS']

def gerar_dataset(qtd_linhas=5000):
    transacoes = []

    for _ in range(qtd_linhas):
        # Escolhe uma loja aleatória
        cat = random.choice(categorias)
        loja = random.choice(base_dados[cat])

        # Gera a descrição suja
        desc = loja
        if random.random() < 0.6: desc = f"{random.choice(sujeira_prefixos)} {desc}"
        if random.random() < 0.4: desc = f"{desc} {random.choice(sujeira_sufixos)}"
        if random.random() < 0.3: desc = f"{desc} {random.randint(10, 9999)}"

        transacoes.append([desc, cat])

    return pd.DataFrame(transacoes, columns=['Descricao', 'Categoria'])

# Executa e salva

print("Gerando 10.000 transações...")
df = gerar_dataset(10000)
df.to_csv('transacoes_train.csv', index=False)

print("Amostra do resultado:")
df

Gerando 10.000 transações...
Amostra do resultado:


,Descricao,Categoria
0,EXTRATO STAR PLUS,ASSINATURAS
1,PIX SALESIANA,EDUCACAO
2,SHOPPING,LOJA DE DEPARTAMENTO
3,COPEL FILIAL 7087,LUZ
4,JOHN JOHN 3339,LOJA DE DEPARTAMENTO
...,...,...
9995,COMPRA VISA AMEPLAN 6070,PLANO DE SAUDE
9996,PIX REVISTA VEJA,ASSINATURAS
9997,TED ANIMALE,LOJA DE DEPARTAMENTO
9998,EXTRATO VR COLLEZIONI 3952,ROUPAS


In [16]:
# Limpa descrições

df = df[~df['Descricao'].str.contains(r'\bPIX\b', regex=True)]
df['Descricao'] = df['Descricao'].str.findall(r'\b(?!\d+\b)\w{2,}\b').str.join(' ').str.upper().str.strip()
for w in sujeira_prefixos + sujeira_sufixos:
    df['Descricao'] = df['Descricao'].str.replace(w, '')

df['Categoria'] = df['Categoria'].str.upper().str.strip()

/tmp/ipython-input-2147818363.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Descricao'] = df['Descricao'].str.findall(r'\b(?!\d+\b)\w{2,}\b').str.join(' ').str.upper().str.strip()
/tmp/ipython-input-2147818363.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Descricao'] = df['Descricao'].str.replace(w, '')
/tmp/ipython-input-2147818363.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X = df['Descricao']
y = df['Categoria']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

model = make_pipeline(
    TfidfVectorizer(),
    RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
)

print("Treinando modelos para todas as colunas...")
model.fit(X_train, y_train)
print("Treino concluído!")

Treinando modelos para todas as colunas...
Treino concluído!


In [18]:
y_pred = model.predict(X_test)

df_final = pd.DataFrame({
    'Descricao': X_test,
    'Real': y_test,
    'Previsto': y_pred
})

probas = model.predict_proba(X_test)
max_probas = np.max(probas, axis=1)
df_final['Confianca'] = max_probas

print(df_final.head(10))

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

                 Descricao                  Real              Previsto  Confianca
7784            FAST SHOP                   CASA                  CASA   0.985793
7023  PROVEDOR DE INTERNET              SERVICOS              SERVICOS   1.000000
2783     MUNDO DAS TINTAS                   CASA                  CASA   0.917347
5074         VERO INTERNET              SERVICOS              SERVICOS   1.000000
9126            DROGA RAIA          MEDICAMENTOS          MEDICAMENTOS   1.000000
2647                  GRF               IMPOSTOS              IMPOSTOS   1.000000
8846    AMAZON PRIME VIDEO           ASSINATURAS           ASSINATURAS   1.000000
7837     MULTA DE TRANSITO                 TAXAS                 TAXAS   1.000000
9290           MANIPULACAO          MEDICAMENTOS          MEDICAMENTOS   1.000000
1201         VISA POLISHOP  LOJA DE DEPARTAMENTO  LOJA DE DEPARTAMENTO   0.990000

Relatório de Classificação:
                      precision    recall  f1-score   support

      

In [19]:
model.fit(X, y)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('randomforestclassifier',
                 RandomForestClassifier(class_weight='balanced',
                                        random_state=42))])

In [20]:
df = pd.read_excel('teste.xlsx')
df['Descricao'] = df['Descricao'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df['Categoria'] = df['Categoria'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

df = df[~df['Descricao'].str.contains(r'\bPIX\b', regex=True)]
df = df[~df['Categoria'].str.contains(r'\bOUTROS\b', regex=True)]
df['Descricao'] = df['Descricao'].str.findall(r'\b(?!\d+\b)\w{2,}\b').str.join(' ').str.upper().str.strip()
for w in sujeira_prefixos + sujeira_sufixos:
    df['Descricao'] = df['Descricao'].str.replace(w, '')

y_pred = model.predict(df['Descricao'])

df_final = pd.DataFrame({
    'Descricao': df['Descricao'],
    'Real': df['Categoria'],
    'Previsto': y_pred
})

probas = model.predict_proba(df['Descricao'])
max_probas = np.max(probas, axis=1)
df_final['Confianca'] = max_probas

print(df_final)

print("\nRelatório de Classificação:")
print(classification_report(df['Categoria'], y_pred))

                                          Descricao                   Real              Previsto  Confianca
0                                    PADARIA MONACO           SUPERMERCADO           RESTAURANTE   0.910000
3                                         MED FARMA           MEDICAMENTOS          MEDICAMENTOS   1.000000
4                                      MEUMARKET24H           SUPERMERCADO          SUPERMERCADO   1.000000
5                              EMPORIO VILA NATURAL           SUPERMERCADO          SUPERMERCADO   0.510000
6                                      MEUMARKET24H           SUPERMERCADO          SUPERMERCADO   1.000000
8      PAGTO ELETRON COBRANCA SUPERGASBRAS ENERGIA                     GAS                   GAS   1.000000
9                                       ATACADAO AS           SUPERMERCADO          SUPERMERCADO   0.960000
10                      CONTA DE LUZ BRADESCO CPFL                     LUZ                   LUZ   0.950000
11                          

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_